In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import json
import random
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction, DataCollatorWithPadding, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [ ]:
# json_path = "/content/drive/MyDrive/Agent/output_cat.jsonl"
# df = pd.read_json(json_path, lines=True)

# data = [{"text": df["messages"][i][1]["content"], "label": df["messages"][i][2]["content"]} for i in range(len(df))]
# with open("/content/drive/MyDrive/Agent/output_cat.json", "w") as file:
#     json.dump(data, file)

In [ ]:
json_path = "/content/drive/MyDrive/Agent/output_cat.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df)
df["label"] = df["label"].map({"Receipt":0, "Promotion":1, "Education":2, "Notice":3, "Career Development":4, "News":5})
dataset = Dataset.from_pandas(df)

                                                   text      label
0     Email content:\nKaggle launched the FGVC12 202...  Education
1     Email content:\nKaggle's BirdCLEF+ 2025 compet...  Education
2     Email content:\nYou received a notification fr...       News
3     Email content:\nKaggle announces the launch of...       News
4     Email content:\nYou purchased "GEAR UP SPECIAL...    Receipt
...                                                 ...        ...
1404  Email content:\nAlan Walker is en route to Aus...       News
1405  Email content:\nTwitter has shared 35 new upda...       News
1406  Email content:\nYou have received 12 notificat...     Notice
1407  Email content:\nTwitter wishes Jasper a bright...       News
1408  Email content:\nYou have received 8 Twitter no...     Notice

[1409 rows x 2 columns]


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Map:   0%|          | 0/1409 [00:00<?, ? examples/s]

In [ ]:
print(test_dataset)
print(len(train_dataset[0]['input_ids']))
print(len(train_dataset[1]['input_ids']))

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 282
})
512
512


In [ ]:
!pip uninstall -q -y bitsandbytes
!pip install -q -U bitsandbytes transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 25.4 MB/s eta 0:00:00


In [ ]:
# training

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True)

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    # quantization_config=bnb_config,
    num_labels=6,
    device_map="auto",
    max_memory={0: "12GB", "cpu": "16GB"}
)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.1, target_modules=["q_proj", "v_proj"], bias="none")

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./qlora_deepseek",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    label_names=["labels"],
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    report_to="tensorboard",
    logging_dir="./logs",
    logging_steps=1,
    eval_steps=1,
    fp16=True,
    optim="adamw_torch",
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    eval_accuracy = accuracy_score(labels, preds)
    return {"accuracy": eval_accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,179,072 || all params: 1,545,902,592 || trainable%: 0.1410


Epoch,Training Loss,Validation Loss,Accuracy
1,0.266200,0.759591,0.748227
2,0.279500,0.406109,0.890071
3,0.021200,0.376244,0.897163
4,0.006200,0.391314,0.886525
5,0.004200,0.424945,0.900709
6,0.000900,0.540795,0.879433
7,0.000500,0.420268,0.907801
8,0.000600,0.519664,0.893617
9,0.000500,0.496635,0.904255
10,0.000100,0.518588,0.900709


TrainOutput(global_step=1400, training_loss=0.10638200921119798, metrics={'train_runtime': 5102.5034, 'train_samples_per_second': 4.417, 'train_steps_per_second': 0.274, 'total_flos': 8.962930123525325e+16, 'train_loss': 0.10638200921119798, 'epoch': 19.72340425531915})

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.42026832699775696, 'eval_accuracy': 0.9078014184397163, 'eval_runtime': 26.8496, 'eval_samples_per_second': 10.503, 'eval_steps_per_second': 2.644, 'epoch': 19.72340425531915}


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Agent")

In [ ]:
# # load the model

# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Agent/finetune_deepseek_classification_2")
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Agent/finetune_deepseek_classification_2")